MIT License

Copyright 2025 James Guana

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

This work has been inspired by: https://www.kaggle.com/code/yashgoyal1605/recomendation-system


# Instruction

*   Test this google collab for compatibility
*   Note that this code can run on ordinary Google Collab with CPU

# Setup Development Environment

In [1]:
!pip install langchain-ollama==0.2.1 langchain-community scikit-learn numpy pandas langchain-core langsmith faiss-cpu mlflow
!apt-get install pciutils lshw
!curl -fsSL https://ollama.com/install.sh | sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.1/415.1 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.7/648.7 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9

In [2]:
# need to run twice as Collab Seem to Fail to Run OLLAMA
import os
import subprocess
from langchain_ollama import OllamaEmbeddings

process = subprocess.Popen("ollama serve", shell=True)
!ollama pull qwen2.5:1.5b

embeddings = OllamaEmbeddings(
    model="qwen2.5:1.5b",
)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 183715c43589...   0% ▕▏    0 B/986 MB                  pulling manifest 
pulling 183715c43589...   0% ▕▏    0 B/986 MB                  pulling manifest 
pulling 183715c43589...   0% ▕▏ 2.3 MB/986 MB                  pulling manifest 
pulling 183715c43589...   3% ▕▏  33 MB/986 MB                  pulling manifest 
pulling 183715c43589...   5% ▕▏  49 MB/986 MB                  pulling manifest 
pulling 183715c43589...   8% ▕▏  74 MB/986 MB                  pulling manifest 
pulling 183715c43589...  10% ▕▏ 101 MB/986 MB                  pulling manifest 
pulling 183715c43589...  12% ▕▏ 115 MB/986 MB                  pulling manifest 
pulling 183715c43589...  15% ▕▏ 147 MB/986 MB                  pulling manifest 
pulling 183715c43589...  19% ▕▏ 189 MB/986 MB                  pulling manifest 
pulling 183715c43589...  20% ▕▏ 198 MB/986 MB  198 MB/s   

In [ ]:
!pip install --upgrade google-api-python-client oauth2client

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import os

auth.authenticate_user()
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()
drive_service = build('drive', 'v3', credentials=credentials)

def download_files_in_folder(service, folder_id, parent_dir):
    query = f"'{folder_id}' in parents and trashed=false"
    results = service.files().list(q=query).execute()
    items = results.get('files', [])

    for item in items:
        file_id = item['id']
        file_name = item['name']
        file_path = os.path.join(parent_dir, file_name)
        if item['mimeType'] == 'application/vnd.google-apps.folder':
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            download_files_in_folder(service, file_id, file_path)
        else:
            print(f'Downloading {file_name} to {file_path}')
            request = service.files().get_media(fileId=file_id)
            with io.FileIO(file_path, 'wb') as fh:
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while done is False:
                    status, done = downloader.next_chunk()
                    print(f'Download {int(status.progress() * 100)}%.')

folder_id = ''
parent_dir = '/content/'

download_files_in_folder(drive_service, folder_id, parent_dir)

print('Downloaded files:')
!ls -R /content/


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 47.2 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.160.0
    Uninstalling google-api-python-client-2.160.0:
      Successfully uninstalled google-api-python-client-2.160.0
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 85%.
Download 100%.
Download 100%.
Download 100%.
Downloaded files:
/content/:
flipkart_com-ecommerce_sample.csv  mlruns  recommend.py  sample_data  utils.py

/content/mlruns:
889985404091409731

/content/mlruns/889985404091409731:
73f09d9d89274e0b96a0d4baa55f524c  meta.yaml

/content/mlruns/889985404091409731/73f09d9d89274e0b96a0d4baa55f524c:
artifacts  meta.yaml  metrics  tags

/content/mlruns/889985404091409731/73f09d9d89274e0b96a0d4baa55f524c/artifacts:
faiss_index

/content/mlruns/889985404091409731/73f09d9d89274e0b96a0d4baa55f524c/arti

# Extract Transform Data

In [4]:
import kagglehub
import pandas as pd
from utils import text_clean

# path = kagglehub.dataset_download("PromptCloudHQ/flipkart-products")

# print("Path to dataset files:", path)

df = pd.read_csv('flipkart_com-ecommerce_sample.csv')
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [5]:
from sklearn.model_selection import train_test_split

print (df.isnull().sum())
df = df.dropna(subset=['description', 'product_specifications'])
df["combined_text"] = df["product_name"] + " " + df["description"] + " " + df["product_specifications"]
df["combined_text"] = df["combined_text"].apply (text_clean)
metadatas = df[["product_name", "product_category_tree", "discounted_price", "brand", "pid"]].to_dict (orient="records")

df_train, df_test = train_test_split (df, test_size = 0.2)
metadatas_train = df_train[["product_name", "product_category_tree", "discounted_price", "brand", "pid"]].to_dict (orient="records")

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64


<ipython-input-5-44936624f2c5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["combined_text"] = df["product_name"] + " " + df["description"] + " " + df["product_specifications"]


# Setup Large Language Model (LLM)

In [6]:
# Setup LLM
#!curl -fsSL https://ollama.com/install.sh | sh
import os
import subprocess
from langchain_ollama import OllamaEmbeddings

process = subprocess.Popen("ollama serve", shell=True)
!ollama pull qwen2.5:1.5b

embeddings = OllamaEmbeddings(
    model="qwen2.5:1.5b",
)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 183715c43589... 100% ▕▏ 986 MB                         
pulling 66b9ea09bd5b... 100% ▕▏   68 B                         
pulling eb4402837c78... 100% ▕▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕▏  11 KB                         
pulling 377ac4d7aeef... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


# Train/Validation of Recommendation Engine

In [ ]:
# run only if recreating the vectorstore
import mlflow
from langchain.vectorstores import FAISS
from recommend import  RECOMMEND

with mlflow.start_run() as run:

    run_id = run.info.run_id

    print(f"Run ID: {run_id}")

    model = RECOMMEND (data = df_train, metadatas=metadatas_train, embeddings=embeddings)

    model.train ()

    vectorstore = model.get_vectorstore ()
    vectorstore.save_local ("faiss_index")

    score = model.score (df_test) # test_df
    print (score)

    mlflow.log_metric("score", score)

    mlflow.log_artifacts("faiss_index", artifact_path="faiss_index")

In [ ]:
model.predict (df=df, id="SFFEGEZCEJRG9HVF")

[GIN] 2025/03/07 - 05:24:25 | 200 |   27.204682ms |       127.0.0.1 | POST     "/api/embed"


,pid,product_name,product_category_tree,discounted_price,brand
0,SFFEGEZ5FYEMHHQW,Gasser Slippers,"[""Footwear >> Men's Footwear >> Slippers & Fli...",299.0,Gasser
1,SFFEJNHJRGMQNRFP,Manthan Slippers,"[""Footwear >> Men's Footwear >> Slippers & Fli...",599.0,Manthan
2,SFFEGJHUECHGZMZS,FAX Slippers,"[""Footwear >> Men's Footwear >> Slippers & Fli...",149.0,FAX
3,SHOEG5XHVBSU5BRW,Niremo Loafers,"[""Footwear >> Women's Footwear >> Casual Shoes...",499.0,Niremo
4,SFFEGG74CMKFQVHF,Walkline Slippers,"[""Walkline Slippers""]",435.0,Walkline
5,SHOEJBV8FAV6DNWW,Rockshose Slip On,"[""Footwear >> Men's Footwear >> Formal Shoes >...",899.0,Rockshose
6,SFFEHVAVYGCD3HSD,Guardian Flip Flops,"[""Footwear >> Men's Footwear >> Slippers & Fli...",289.0,Guardian
7,SFFEJT5THKSESHQJ,Krocs Slippers,"[""Footwear >> Women's Footwear >> Slippers & F...",449.0,Krocs
8,SNDEJGYFP2AT3FMH,Style HD Men Sandals,"[""Footwear >> Men's Footwear >> Sandals & Floa...",399.0,Style HD


# Train Recommendation Engine for Deployment (All Data)

In [ ]:
# deployment training

import mlflow
from langchain.vectorstores import FAISS
from recommend import  RECOMMEND

mlflow.set_experiment("deploy")

with mlflow.start_run() as run:

    run_id = run.info.run_id

    print(f"Run ID: {run_id}")

    model = RECOMMEND (data = df, metadatas=metadatas, embeddings=embeddings)

    model.train ()

    vectorstore = model.get_vectorstore ()
    vectorstore.save_local ("faiss_index")

    score = model.score (df_test) # test_df
    print (f"Model: {score}")

    mlflow.log_metric("score", score)

    mlflow.log_artifacts("faiss_index", artifact_path="faiss_index")

2025/03/07 05:25:57 INFO mlflow.tracking.fluent: Experiment with name 'deploy' does not exist. Creating a new experiment.


Run ID: 73f09d9d89274e0b96a0d4baa55f524c
[GIN] 2025/03/07 - 05:30:12 | 200 |         4m14s |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   32.621534ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   25.820437ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   29.564318ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   27.775126ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   30.628926ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   34.942926ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   23.198013ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   32.190233ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   28.733946ms |       127.0.0.1 | POST     "/api/embed"
[GIN] 2025/03/07 - 05:30:17 | 200 |   2

# Load/Initialize Recommendation Engine

In [7]:
import mlflow
from langchain.vectorstores import FAISS
import kagglehub
import pandas as pd
from utils import text_clean

import subprocess
from langchain_ollama import OllamaEmbeddings
from recommend import  RECOMMEND

def load_faiss_index(run_id, embeddings):
    local_dir = mlflow.artifacts.download_artifacts(run_id=run_id)
    return FAISS.load_local(local_dir+'/'+"faiss_index", embeddings, allow_dangerous_deserialization=True)

process = subprocess.Popen("ollama serve", shell=True)
!ollama pull qwen2.5:1.5b

embeddings = OllamaEmbeddings(
    model="qwen2.5:1.5b",
)

df = pd.read_csv('flipkart_com-ecommerce_sample.csv')
df.head()

df = df.dropna(subset=['description', 'product_specifications'])
df["combined_text"] = df["product_name"] + " " + df["description"] + " " + df["product_specifications"]
df["combined_text"] = df["combined_text"].apply (text_clean)
metadatas = df[["product_name", "product_category_tree", "discounted_price", "brand", "pid"]].to_dict (orient="records")

vectorstore = load_faiss_index("73f09d9d89274e0b96a0d4baa55f524c", embeddings)
model = RECOMMEND (data = df, metadatas=metadatas, embeddings=embeddings)
model.set_vectorstore (vectorstore=vectorstore)

pulling manifest ⠋ pulling manifest ⠙ pulling manifest 
pulling 183715c43589... 100% ▕▏ 986 MB                         
pulling 66b9ea09bd5b... 100% ▕▏   68 B                         
pulling eb4402837c78... 100% ▕▏ 1.5 KB                         
pulling 832dd9e00a68... 100% ▕▏  11 KB                         
pulling 377ac4d7aeef... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


# Get Recommendations

In [8]:
# Get Recommendations
# Note that the result will be less than 1 as the similarity_search also includes the source PID, so the source PID is removed by the code
model.set_k (20) # set number of recommendations to show. note that recommendations farther from zero are become less and less similar to the target product
model.predict (df=df, id="SRTEH2FF9KEDEFGF") # PID of target product

,pid,product_name,product_category_tree,discounted_price,brand
0,SRTEH2FGBDJGX8FW,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",379.0,Alisha
1,SRTEH2FECMGNZJXJ,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",379.0,Alisha
2,SRTEH2F6HUZMQ6SJ,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",267.0,Alisha
3,SRTEH2FVUHAAVH9X,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",479.0,Alisha
4,SRTEHH8886VHZQAM,Lilliput Solid Boy's Bermuda Shorts,"[""Clothing >> Kids' Clothing >> Boys Wear >> S...",1236.0,Lilliput
5,SRTEHVUP4FPGWYVZ,FS Mini Klub Printed Baby Girl's Basic Shorts,"[""Baby Care >> Infant Wear >> Baby Girls' Clot...",499.0,FS Mini Klub
6,SRTEGWKJQDFFN5E5,"Mynte Solid Women's Cycling Shorts, Gym Shorts...","[""Clothing >> Women's Clothing >> Sports & Gym...",649.0,Mynte
7,SRTEGWKKZNUYX7HW,"Mynte Solid Women's Cycling Shorts, Gym Shorts...","[""Clothing >> Women's Clothing >> Sports & Gym...",649.0,Mynte
8,TSHEGNVDSSTAG8C3,TSG Escape Printed Men's Round Neck Multicolor...,"[""Clothing >> Men's Clothing >> T-Shirts >> TS...",449.0,TSG Escape
9,SRTEGWKKXGGSERKZ,"Mynte Solid Women's Cycling Shorts, Gym Shorts...","[""Clothing >> Women's Clothing >> Sports & Gym...",649.0,Mynte
